### Resources
#### https://catalog.data.gov/dataset/census-api-by-coordinates
#### https://www.programmableweb.com/category/all/apis?keyword=zip%20code
#### https://www.census.gov/geo/maps-data/data/geocoder.html
#### https://www.census.gov/data/developers/data-sets/popest-popproj.html
#### https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf

### zip
#### Cleveland: 44114
#### LA: 90071
#### Austin: 78705
#### NYC: 10019
#### Denver: 80202

In [1]:
#import libraries and api keys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import requests
import json
from urllib import 
#from keys.gitignore import 
#from keys import 

# Gather zip codes for cities
### Build lat/lng list for the 5 cities
### Use Radius by Zip API to return zip codes inside each radius
####      Math to remove inner ring from current calculation
### Build zip code list for each city/radius

In [233]:
# Create a dictionary of the cities' names and their zipcodes
cities = {'Cleveland':44114,
          'LA': 90071,
          'Austin': 78705,
          'NYC': 10019,
          'Denver': 80202
         }
# APi key for https://www.zipcodeapi.com/API
api = "F1Q5equ8jewDKQnEnJ5o2SYzmUj0WJr7wjbvYlHLG1xlGqrc5XNzVYYRC4RB3cT1"

# Loop through the cities to get the required zip_code
for city in cities:
    # Create an api request from the website 
    data_1 = requests.get(f"https://www.zipcodeapi.com/rest/F1Q5equ8jewDKQnEnJ5o2SYzmUj0WJr7wjbvYlHLG1xlGqrc5XNzVYYRC4RB3cT1/radius.csv/{cities[city]}/20/mile")
    
    # Create an empty list to save the results later
    zipcode_data = []
    # Get the text as str for further processing and slpit them
    text = data_1.text
    for row in text.split('\n'):
        zipcode_data.append(row.split(','))
    
    # Create a data Frame for the results collected from preivous step
    df = pd.DataFrame(zipcode_data)
    #Set the columns index 
    df = df.set_axis(['zip_code','distance','city','state'], axis='columns', inplace=False)
    # Drop the first row which contains unnecessarily string data
    df = df.drop(0)
    # Convert to a numeric data from distance
    df['distance'] = pd.to_numeric(df['distance'])
    # Sort the values by ascending order
    df = df.sort_values(by='distance', ascending=True)
    # Save it to a csv file by the city name
    df.to_csv(f'{city}.csv')

# Convert zip coces to ZTAC
## Use Census API to return ZTAC for each zip code in list

# Lookup ZTAC value against median age in CSV

# Create dataframe

# Analysis